## Prueba NG Solve

Simulación

Primero se debe llamar a la gui (graphical user interphace), así esta se abre y luego cualquier simulación se observará en ella

In [31]:
# use netgen.meshing module for 1d meshes
from ngsolve import *
#from netgen.meshing import Mesh as Mesh1d
#from netgen.meshing import MeshPoint,Element0D, Element1D
from netgen.csg import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib as plt
from math import pi
from netgen.geom2d import SplineGeometry
from ngsolve.krylovspace import CGSolver

In [72]:
def initial_conds(num):
    if num==1:
        v0 = 1 + 0.5*cos(4*pi*x) * cos(4*pi*y)
        w0 = 1 + 0.5*cos(8*pi*x) * cos(8*pi*y)
        T = 3
        D = 1
        a = 0.2232
        b = 0.9
        lamb = -1
        thet = 0.004
        params = (T, D, a, b, lamb, thet)
        I_app = 0
        h = 1/32; CFL = 40 / 32
        
    
    elif num==2:
        v0 = 1 - 1 / (1 + exp(-50 * sqrt(x**2 + y**2) - 0.1))
        w0 = 0
        T = 10
        D = 1
        a = 0.16875
        b = 1.0
        lamb = -100
        thet = 0.25
        params = (T, D, a, b, lamb, thet)
        I_app = 0
        h = 1/32; CFL = 40 / 32
        
        
    return v0, w0, params, I_app, h, CFL

def solve_FHN1(v0, w0, params, I_app=0, h=0.1, CFL=0.3, order=3, ion=False, DG=False):
    """
    Usa euler explicito.
    """
    T, D, a, b, lamb, thet = params
    
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))  
    if DG:
        V = L2(mesh, order=order) # espacio de funciones integrables para 1ra ecuacion
        Q = L2(mesh, order=order) # espacio de funciones integrables para 2da ecuacion
    else:
        V = H1(mesh, order=order) # espacio de funciones con derivada integrable para 1ra ecuacion
        Q = H1(mesh, order=order) # espacio de funciones con derivada integrable para 2da ecuacion
    
    X = V*Q # espacio producto
    (v, w), (p, q) = X.TnT() # funciones trial y test


    c = BilinearForm(X, nonassemble=True)
    c += (D * grad(v) * grad(p)) * dx # primera ecuacion fitshugh-nagumo estatica (sin tiempo)
    if ion: 
        I_ion = (-lamb*v*(v - thet)*(1 - v)*p + lamb*w*p) * dx # termino no lineal (corriente ionica)
        c += I_ion
    c += (b*w*q - a*v*q)*dx # segunda ecuacion fitshugh-nagumo estatica # No funciona hacer v*q
    c.Assemble()

    if I_app:
        l = LinearForm(X)
        l += I_app * p * dx
        l.Assemble()

    m = X.Mass(1)
    minv = m.Inverse()
    
    #elvol = Integrate(CoefficientFunction(1), mesh, element_wise=True)
    #h = max([(2*vol)**(1/2) for vol in elvol]) # 2D
    t = 0; dt = CFL*h

    gfu = GridFunction(X)
    gfu.components[0].Set(v0) # Se fija condicion inicial v
    gfu.components[1].Set(w0) # Se fija condicion inicial w

    sceneu = Draw(gfu.components[0], mesh, "v", autoscale=True, animate=False, order=5)

    res = gfu.vec.CreateVector()
    
    v_sol = [v0]; w_sol = [w0]

    while t + dt/2 <= T:
        res.data = minv @ c.mat * gfu.vec
        gfu.vec.data -= dt * res
        t += dt
        #v_sol.append(gfu.components[0].data); w_sol.append(gfu.components[1].data)
        sceneu.Redraw()
        
    return gfu, v_sol, w_sol

In [73]:
v0, w0, params, I_app, h, CFL = initial_conds(1)
gfu = solve_FHN1(v0, w0, params, I_app=I_app, h=h, CFL=CFL, DG=False, ion=True)


craete bilinearformapplication


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE

SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE

SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE
SolveM is only available for L2-space, not for N6ngcomp18H1HighOrderFESpaceE